## Model with PyTorch

by: Andreas Chandra \
date: 21/03/2021

In [83]:
import time
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

In [2]:
d_data = pd.read_csv("../data/clean/df_41017.csv")

In [3]:
swh_arr = d_data.swh.values

In [4]:
dataset = np.zeros((32899, 6))

In [5]:
for idx, start in enumerate(range(0, len(swh_arr) - 5)):
    row = swh_arr[start:start+6]
    dataset[idx, :] = row

In [6]:
dataset_nona = dataset[~np.isnan(dataset).any(axis=1)]

In [7]:
X = dataset_nona[:, :5]
y = dataset_nona[:, 5]

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [69]:
class OceanDataset(Dataset):
    def __init__(self, X_train, X_test, y_train, y_test):
        self.dataset = {
            'train': (X_train, y_train, len(X_train)),
            'test': (X_test, y_test, len(y_test))
        }
        
        self.set_split('train')
        
    def set_split(self, split='train'):
        self.X, self.y, self.length = self.dataset[split]
        
    def __getitem__(self, idx):
        x = torch.FloatTensor(self.X[idx])
        y = torch.FloatTensor([self.y[idx]])
        
        return x, y
    
    def __len__(self):
        return self.length    

In [88]:
class Regressor(nn.Module):
    def __init__(self,):
        super(Regressor, self).__init__()
        self.archi = nn.Sequential(
            nn.Linear(in_features=5, out_features=15),
            nn.Linear(in_features=15, out_features=1)
        )
        
    def forward(self, input_):
        out = self.archi(input_)
        out = torch.relu(out)
        
        return out

In [89]:
dataset = OceanDataset(X_train, X_test, y_train, y_test)

In [90]:
model = Regressor()

In [91]:
num_params = sum(p.numel() for p in model.parameters())
print(f"Total trainable parameters {num_params}")

Total trainable parameters 106


In [92]:
optimizer = optim.Adam(model.parameters(), lr=1e-4)
criterion = nn.MSELoss()

#### Training Routine

In [93]:
for epoch in range(1, 51):
    
    loss_train = 0
    loss_val = 0
    
    start = time.time()
    
    model.train()
    dataset.set_split('train')
    data_gen = DataLoader(dataset, batch_size=32)
    for batch_index, (x, y) in enumerate(data_gen, 1):
        optimizer.zero_grad()
        
        out = model(x)
        
        loss = torch.sqrt(criterion(out, y))
        loss_train += (loss.item() - loss_train) / batch_index
        loss.backward()
        
        optimizer.step()
    
    model.eval()
    dataset.set_split('test')
    data_gen = DataLoader(dataset, batch_size=32)
    for batch_index, (x, y) in enumerate(data_gen, 1):
        with torch.no_grad():
            out = model(x)
            
            loss = torch.sqrt(criterion(out, y))
            loss_val += (loss.item() - loss_val) / batch_index
    
    duration = time.time() - start
    
    print(f"epoch {epoch} | {duration:.2f}s")
    print(f"train loss: {loss_train:.2f} val loss: {loss_val:.2f}")

epoch 1 | 2.48s
train loss: 0.50 val loss: 0.15
epoch 2 | 2.40s
train loss: 0.15 val loss: 0.14
epoch 3 | 2.31s
train loss: 0.13 val loss: 0.12
epoch 4 | 2.38s
train loss: 0.12 val loss: 0.12
epoch 5 | 2.37s
train loss: 0.12 val loss: 0.12
epoch 6 | 2.37s
train loss: 0.12 val loss: 0.12
epoch 7 | 2.40s
train loss: 0.12 val loss: 0.11
epoch 8 | 2.43s
train loss: 0.12 val loss: 0.11
epoch 9 | 2.38s
train loss: 0.11 val loss: 0.11
epoch 10 | 2.35s
train loss: 0.11 val loss: 0.11
epoch 11 | 2.43s
train loss: 0.11 val loss: 0.11
epoch 12 | 2.35s
train loss: 0.11 val loss: 0.11
epoch 13 | 2.38s
train loss: 0.11 val loss: 0.11
epoch 14 | 2.41s
train loss: 0.11 val loss: 0.11
epoch 15 | 2.41s
train loss: 0.11 val loss: 0.11
epoch 16 | 2.35s
train loss: 0.11 val loss: 0.11
epoch 17 | 2.37s
train loss: 0.11 val loss: 0.11
epoch 18 | 2.38s
train loss: 0.11 val loss: 0.11
epoch 19 | 2.39s
train loss: 0.11 val loss: 0.11
epoch 20 | 2.39s
train loss: 0.11 val loss: 0.11
epoch 21 | 2.43s
train loss: 